In [ ]:
# Please do not change this cell because some hidden tests might depend on it.
import os

# Otter grader does not handle ! commands well, so we define and use our
# own function to execute shell commands.
def shell(commands, warn=True):
    """Executes the string `commands` as a sequence of shell commands.
     
       Prints the result to stdout and returns the exit status. 
       Provides a printed warning on non-zero exit status unless `warn` 
       flag is unset.
    """
    file = os.popen(commands)
    print (file.read().rstrip('\n'))
    exit_status = file.close()
    if warn and exit_status != None:
        print(f"Completed with errors. Exit status: {exit_status}\n")
    return exit_status

shell("""
ls requirements.txt >/dev/null 2>&1
if [ ! $? = 0 ]; then
 rm -rf .tmp
 git clone https://github.com/cs236299-2020/lab1-1.git .tmp
 mv .tmp/tests ./
 mv .tmp/requirements.txt ./
 rm -rf .tmp
fi
pip install -q -r requirements.txt
""")

In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook()

# Course 236299
## Lab 1-1 – Types, tokens, and representing text

In [ ]:
import math
import numpy as np
import re
import sys
import torchtext

Where we're headed: Nearest neighbor text classification works by classifying a novel text with the same class as that of the text that is closest according to some distance metric. These metrics are calculated based on representations of the texts. In this lab, we'll introduce some different representations and you'll use nearest neighbor classification to predict the speaker of sentences selected from a children's book.
    
The objectives of this lab are to:

* Clarify terminology around words and texts,
* Manipulate different representations of words and texts,
* Apply these representations to calculate text similarity, and
* Classify documents by a simple nearest neighbor model.
   
In this and later labs, we have you carry out several exercises in notebook cells. The cells you are to do are marked '`#TODO`'. They will typically have a `...` where your code or answer should go. Where specified, you needn't write code to calculate the answer, but instead, simply work out the answer yourself and enter it.

New bits of Python used for the first time in the _solution set_ for this lab, and which you may therefore find useful:

* `len`
* `math.pi`
* `numpy.dot`
* `numpy.linalg.norm`
* `numpy.maximum`
* `numpy.minimum`
* `numpy.sum`
* `numpy.where`
* `numpy.zeros`
* `numpy.zeros_like`
* `re.match`
* `str.join`
* `str.lower`
* `torchtext.data.get_tokenizer`

## Counting words

<img src="https://github.com/nlp-236299/data/blob/master/Seuss/seuss%20-%201966%20-%20green%20eggs%20and%20ham.gif?raw=true" width=150 align=right />

Here are five sentences from Dr. Seuss's [_Green Eggs and Ham_](https://en.wikipedia.org/wiki/Green_Eggs_and_Ham):

    Would you like them here or there?
    I would not like them here or there.
    I would not like them anywhere.
    I do not like green eggs and ham.
    I do not like them, Sam-I-am.

We'll make this text available in the variable `text`.

In [ ]:
text = """
    Would you like them here or there?
    I would not like them here or there.
    I would not like them anywhere.
    I do not like green eggs and ham.
    I do not like them, Sam-I-am.
    """

A Python string like this is, of course, a sequence of characters. But we think of this text as a sequence of sentences each composed of a sequence of words. How many words are there in this text? That is a fraught question, for several reasons, including

* The type-token distinction
* Tokenization issues
* Normalization

## Types versus tokens

In determining the number of words in `text`, are we talking about word _types_ or word _tokens_. (For instance, there are five _tokens_ of the word _type_ 'like'.)

How many word tokens are there in total in this text?
Write code that counts them, or count them manually.
Assign the number to the variable `token_count` in the next cell.
<!--
BEGIN QUESTION
name: token_count
-->
> If you have the otter-grader module properly installed (`pip install otter-grader`), you can test your solution using this unit test by running the `grader.check`("token_count") two cells below. There's also a cell at the end of the notebook to run all of the unit tests.

In [ ]:
#TODO - define `token_count` to be the number of tokens in `text`
token_count = ...

In [ ]:
grader.check("token_count")

How many word types are there? (Again, you can just count manually.)
<!--
BEGIN QUESTION
name: type_count
-->

In [ ]:
#TODO - define `type_count` to be the number of types in `text`
type_count = ...

In [ ]:
grader.check("type_count")

<!-- BEGIN QUESTION -->

The set of types of a language is referred to as its _vocabulary_. Are there more types or tokens as you calculated above? Could it be otherwise?
<!--
BEGIN QUESTION
name: type_vs_token_count
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Tokenization 

Did you count 'there?' as one token or two? This raises the issue of _tokenization_ of text, how to decide where the token boundaries occur. For instance, here's a simple way to split a string – to _tokenize_ it – in Python by splitting at whitespace.

In [ ]:
def whitespace_tokenize(str):
    return str.split()

Try it out on the `text` defined above.
<!--
BEGIN QUESTION
name: tokens_whitespace
-->

In [ ]:
#TODO - define `tokens` to be the tokens as defined by the `whitespace_tokenize` function
tokens = ...

In [ ]:
grader.check("tokens_whitespace")

Using this tokenization method, count the number of tokens in the text, this time using Python to do the work.
<!--
BEGIN QUESTION
name: token_count_whitespace
-->

In [ ]:
#TODO - place your token count here
token_count_2 = ...

In [ ]:
grader.check("token_count_whitespace")

Arguably, we _should_ split off punctuation as separate tokens, but even there, some care must be taken. We don't want to split 'don't' into three tokens or 'Sam-I-Am' into five. (There's a good argument to be made however that the string 'don't' should be construed as two tokens, namely, 'do' and 'n't', but that's beyond the scope of today's discussion.)

Here, we provide an alternative tokenizer that splits tokens at whitespace and splits off punctuation at the beginning and end of non-whitespace regions as separate tokens as well. It makes use of [the Python `re` module](https://docs.python.org/3/library/re.html) for regular expressions to specify the splitting process. Look over the code and make sure you understand what's going on.

In [ ]:
def punc_tokenize(str):
    return [tok for tok in re.split('(\W*?)\s+(\W*)', str) if tok != '']

Now how many tokens are there in the text if tokenized in this way?
<!--
BEGIN QUESTION
name: token_count_punc
-->

In [ ]:
#TODO
token_count_3 = ...

In [ ]:
grader.check("token_count_punc")

## Normalization

This tokenization method counts 'Would' and 'would' (capitalized and uncapitalized) as separate types. Is that a good idea? This raises the issue of text _normalization_.

Define a function `normalize_token` that normalizes tokens by making them lowercase if at most the first character is uppercase. (Hints [here](https://docs.python.org/3/library/stdtypes.html#str.lower) and [here](https://docs.python.org/3/library/re.html#re.match). These are also listed in the hint cell at the top of the lab, so we'll mostly stop providing these hints from here on.)
<!--
BEGIN QUESTION
name: normalize_token
-->

In [ ]:
#TODO - implement normalize_token, which returns the normalized word for a single word `str`
def normalize_token(str):
    ...

In [ ]:
grader.check("normalize_token")

Now define `norm_tokens_punc` to be the sequence of normalized tokens as tokenized by `punc_tokenize`
<!--
BEGIN QUESTION
name: norm_tokens_punc
-->

In [ ]:
#TODO
norm_tokens_punc = ...

In [ ]:
grader.check("norm_tokens_punc")

How many types are there when tokenized and normalized in this way?
<!--
BEGIN QUESTION
name: type_count_norm_punc
-->

In [ ]:
#TODO
type_count_norm_punc = ...

In [ ]:
grader.check("type_count_norm_punc")

## Using prebuilt tokenizers

Tokenization and normalization are so commonly needed that many packages provide pre-built tokenizers of various sorts. We'll use one from `torchtext`, a package that we'll make a fair amount of use of in the course.

Define two tokenizers, versions of `whitespace_tokenize` and a normalized version of `punc_tokenize` above, using [the `torchtext.data.get_tokenizer` function](https://pytorch.org/text/data.html#get-tokenizer). (Use respectively the `None` and `"basic_english"` tokenizers that they provide.)
<!--
BEGIN QUESTION
name: tt_whitespace_tokenize_and_tt_normpunc_tokenize
-->

In [ ]:
#TODO
def tt_whitespace_tokenize(str):
    ...
    
def tt_normpunc_tokenize(str):
    ...

In [ ]:
grader.check("tt_whitespace_tokenize_and_tt_normpunc_tokenize")

Now we should be able to print out the last few tokens of the sample text, tokenized using these functions.

In [ ]:
print(tt_whitespace_tokenize(text)[-10:])
print(tt_normpunc_tokenize(text)[-10:])

> _Meta-comment:_ Because it's important that you get practice both with implementing the ideas in the course from first principles and also with using prebuilt software that provides similar functionality, we'll often have you engage in this seemingly redundant process of first implementing a small example and then applying a prebuilt method to do much the same thing. The effort may be duplicative, but it is not wasted.

## Representing words

In this section, we'll explore some simple representations for tokens, as a step on the way to representing texts – sentences or documents:

### String encoding
We've already seen string encoding above, representing a token of a word type by a string specific to that type: a token 'green' represented by an instance of the Python string `'green'`, for instance, or 'Sam-I-am' represented by `'Sam-I-am'`. So let's move on.

### 1-hot encoding
Given a vocabulary for a language, we can associate each type with an integer, say by its index in a vector. Notice that we've already imported the `numpy` module under the name `np`; we'll use a `numpy` array for the vocabulary vector. For the Seuss text, we can use this vocabulary:

In [ ]:
vocabulary = np.array(sorted(set(norm_tokens_punc)))
vocabulary

#### A digression on `numpy` arrays

An important property of `numpy` arrays is that many operations on them work componentwise, that is, separately for each component of the array, rather than on the array all at once. This is called [broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html), and it is common in libraries such as `numpy`, `torch`, and `tensorflow`. Compare the following two operations, first on lists, then on `numpy` arrays.

In [ ]:
['once', 'twice'] == 'once'

In [ ]:
np.array(['once', 'twice']) == 'once'

This behavior of `numpy` arrays (and PyTorch tensors, as we will see) is quite powerful, allowing for simply specifying complex operations and for efficient, even parallelizable, computation of them. (If you want a deeper understanding of how `numpy`'s N-dimensional arrays work, see [the array documentation](https://numpy.org/doc/stable/reference/arrays.html).)

But back to the 1-hot representation.

In the _1-hot representation_ of words, a token is then represented by a bit vector (again given as a `numpy` array), with a 1 at the index of the token's type. For instance, the 1-hot representation of the comma token ',' would be

In [ ]:
np.array([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

Conversion back and forth between these various representations is useful. Define functions `str_to_onehot` and `onehot_to_str` that convert between the string and one-hot representations using a vocabulary array to define the  conversion. 

Ideally, in your implementation, you'll want to take advantage of the componentwise nature of many `numpy` operations discussed above.

In [ ]:
#TODO
def str_to_onehot(wordtype, vocabulary):
    """Returns the 1-hot representation of `wordtype`, a string, using 
    `vocabulary`, a string vector, to define the conversion.
    The returned value should be a np.array with data type int.
    """
    ...

def onehot_to_str(onehot, vocabulary):
    """Returns the string representation of `onehot`, a one-hot 
    representation of a word type, using `vocabulary`, a string 
    vector, to define the conversion.
    """
    ...

Now use `str_to_onehot` to define the variable `anywhere_1hot` to be the 1-hot representation for a token of the type 'anywhere'. 
<!--
BEGIN QUESTION
name: anywhere_1hot
-->

In [ ]:
#TODO
anywhere_1hot = ...

In [ ]:
grader.check("anywhere_1hot")

You can verify that the conversion worked correctly by inverting it using `onehot_to_str`, which we've done in the following unit test.
<!--
BEGIN QUESTION
name: anywhere_1hot_reverse
-->

In [ ]:
grader.check("anywhere_1hot_reverse")

## Representing texts

### The set-of-words representation

We can represent a whole text (a sequence of words) by manipulating the vector representations of the words within the text. For instance, we can take the **elementwise maximum** of the vectors. We refer to this as the _set-of-words_ representation.

Here we've defined a function `set_of_words` that returns the set of words representation for a token sequence.

In [ ]:
def set_of_words(tokens, vocabulary):
    """Returns the set-of-words representation as a numpy array for the 
    sequence of `tokens` using the `vocabulary` to specify the conversion.
    """
    onehots = np.array([str_to_onehot(token, vocabulary) for token in tokens])
    return np.amax(onehots, 0)

This representation for a text is a vector that has a `1` for each word type that occurs in the text. The vector represents the subset of vocabulary words that appear in the text; hence the term 'set of words'.


What is the set-of-words representation for the example text 'I would not, would not, here or there.'?
<!--
BEGIN QUESTION
name: example_sow
-->

In [ ]:
#TODO - define the variable to be the set of words representation for the example text
# Use tt_normpunc_tokenize tokenizer
example_sow = ...
    ...

In [ ]:
grader.check("example_sow")

### The bag of words representation

If instead, we take the componentwise _addition_ of the vectors instead of the maximum, the text representation provides the _frequency_ of each word type in the text. We refer to this representation as the _[bag](https://en.wikipedia.org/wiki/Multiset) of words_ representation. 

Define a function `bag_of_words`, analogous to `set_of_words` above, that returns the bag-of-words representation for a token sequence.

In [ ]:
#TODO
def bag_of_words(tokens, vocabulary):
    ...

What is the bag of words representation for the example text 'I would not, would not, here or there.'?
<!--
BEGIN QUESTION
name: example_bow
-->

In [ ]:
#TODO - define the variable to be the bag of words representation for the example text
# Use tt_normpunc_tokenize tokenizer
example_bow = ...

In [ ]:
grader.check("example_bow")

## Document similarity metrics

Consider the following text classification problem: Each sentence in _Green Eggs and Ham_ is spoken by one of two characters, Sam-I-Am and Guy-Am-I. We want to be able to classify new sentences as (most likely) being uttered by one of the two.

A simple method for text classification is the _nearest neighbor_ method. We select the class for the new sentence that is the same as the class of the "nearest" (most similar) sentence for which we already know the class. (You'll experiment much more with this text classification method in the next lab.)

To perform nearest neighbor classification, we need a method for measuring the (metaphorical) distance between two texts based on their representations. We'll explore a few methods here:

* Hamming distance
* Jaccard distance
* Euclidean distance
* cosine distance

You'll implement code for all of these distance metrics. Try to implement the functions using `numpy` vector functions only, without explicit iteration over the elements in the vector.

We'll take a look at the distances among the following sentences:

1. Would you like them here or there?
2. I would not like them here or there.
3. Do you like green eggs and ham?
4. I do not like them Sam-I-Am.

We'll start with the set of words representations of these sentences:

In [ ]:
examples = """Would you like them here or there?
              I would not like them here or there.
              Do you like green eggs and ham?
              I do not like them Sam-I-Am.""" \
           .split("\n")
sows = [set_of_words(tt_normpunc_tokenize(sentence), vocabulary) 
            for sentence in examples]

### Hamming distance

The Hamming distance between two vectors is the number of positions at which they differ. Define a function `hamming_distance` that computes the Hamming distance between two vectors.
<!--
BEGIN QUESTION
name: hamming_distance
-->

In [ ]:
#TODO - implement hamming_distance. The returned value should be a integer.
def hamming_distance(v1, v2):
    ...

In [ ]:
grader.check("hamming_distance")

Now we can generate the Hamming distances among all of the sample sentences in a little table. Do the values make sense?

In [ ]:
for i in range(4):
    for j in range(4):
        print(f"{hamming_distance(sows[i], sows[j]):4} ", end='')
    print()

### Jaccard distance

The Jaccard distance between two sets (and remember that these bit strings basically represent sets) is one minus the number of elements in their intersection divided by the number of elements in their union.

$$ D_{jaccard}(v_1, v_2) = 1 - \frac{| v_1 \cap v_2 |}{| v_1 \cup v_2 |} $$

Define a function `jaccard_distance`.
<!--
BEGIN QUESTION
name: jaccard_distance
-->

In [ ]:
#TODO
def jaccard_distance(v1, v2):
    ...

In [ ]:
grader.check("jaccard_distance")

Again, here's a table of the Jaccard distances among the sample sentences.

In [ ]:
for i in range(4):
    for j in range(4):
        print(f"{jaccard_distance(sows[i], sows[j]):5.3f} ", end='')
    print()

## Euclidean distance

The Euclidean distance between two vectors is the norm of the vector between them, that is,

$$ D_{euclidean}(\mathbf{x}, \mathbf{y}) = |\mathbf{x} - \mathbf{y}| $$

where $|\mathbf{z}|$, the norm of a vector $\mathbf{z}$, is calculated as

$$ |\mathbf{z}| = \sqrt{\sum_{i=1}^N \mathbf{z}_i^2} $$

Fortunately, `numpy` provides the function `np.linalg.norm` to compute the norm, and the vector between two vectors is computed by componentwise subtraction.

Define a function `euclidean_distance` to compute the Euclidean distance between two vectors.
<!--
BEGIN QUESTION
name: euclidean_distance
-->

In [ ]:
#TODO
def euclidean_distance(v1, v2):
    ...

In [ ]:
grader.check("euclidean_distance")

Again, here's a table of the Euclidean distances among the sample sentences.

In [ ]:
for i in range(4):
    for j in range(4):
        print(f"{euclidean_distance(sows[i], sows[j]):5.3f} ", end='')
    print()

## Cosine distance

The _cosine similarity_ of two vectors of length $N$ is the cosine of the angle that they form, which is computed as the dot product of the two vectors divided by their norms.

$$ cos(\mathbf{x}, \mathbf{y}) = 
      \frac{\sum_{i=1}^N \mathbf{x}_i \cdot \mathbf{y}_i}{|\mathbf{x}| \cdot |\mathbf{y}|} $$

This isn't a distance metric, but a similarity metric. For vectors of non-negative numbers, it ranges from 0 to 1, where 0 is maximally different and 1 is maximally similar. To turn it into a distance metric, then, we take the inverse cosine (to convert the cosine to an angle between $\pi$ and 0) and divide by $\pi$.

$$ D_{cosine}(\mathbf{x}, \mathbf{y}) = \frac{cos^{-1}(cos(\mathbf{x}, \mathbf{y}))}{\pi} $$

Since we're using `numpy`, some of these functions are already provided. See hints [here](https://numpy.org/doc/stable/reference/generated/numpy.dot.html), [here](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html?highlight=norm#numpy.linalg.norm), and [here](https://numpy.org/doc/stable/reference/generated/numpy.arccos.html?highlight=arccos#numpy.arccos).

(To avoid some math domain errors, we recommend that you use the function `safe_acos` that we've provided to compute the inverse cosine function instead of using `math.acos` directly.)

In [ ]:
def safe_acos(x):
    """Returns the arc cosine of `x`. Unlike `math.acos`, it 
       does not raise an exception for values of `x` out of range, 
       but rather clips `x` at -1..1, thereby avoiding math domain
       errors in the case of numerical errors."""
    return math.acos(math.copysign(min(1.0, abs(x)), x))

In [ ]:
#TODO
def cosine_distance(v1, v2):
    """Returns the cosine distance between two vectors"""
    ...

In [ ]:
grader.check("cosine_distance")

In [ ]:
for i in range(4):
    for j in range(4):
        print(f"{cosine_distance(sows[i], sows[j]):5.3f} ", end='')
    print()

In the next lab, you'll use some of these distance metrics to automatically classify text using nearest neighbor classification.

<!-- BEGIN QUESTION -->

## Lab debrief – for consensus submission only

**Question:** We're interested in any thoughts your group has about this lab so that we can improve this lab for later years, and to inform later labs for this year. Please list any issues that arose or comments you have to improve the lab. Useful things to comment on include the following: 

* Was the lab too long or too short?
* Were the readings appropriate for the lab? 
* Was it clear (at least after you completed the lab) what the points of the exercises were? 
* Are there additions or changes you think would make the lab better?

<!--
BEGIN QUESTION
name: open_response_debrief
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



# End of lab 1-1

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()